In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

In [ ]:
dt = pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')
dt.head()

In [ ]:
plt.figure(figsize=(20,15))
sns.set_theme(style='dark')
plt.subplot(2,3,1)
sns.countplot(data=dt,x='fbs',hue='target')
plt.subplot(2,3,2)
sns.countplot(data=dt,x='restecg',hue='target')
plt.subplot(2,3,3)
sns.countplot(data=dt,x='slope',hue='target')
plt.subplot(2,3,4)
sns.countplot(data=dt,x='ca',hue='target')
plt.subplot(2,3,5)
sns.countplot(data=dt,x='exang',hue='target')
plt.subplot(2,3,6)
sns.countplot(data=dt,x='thal',hue='target')
plt.show()

In [ ]:
plt.figure(figsize=(13,13))

sns.set_theme(style='darkgrid')
plt.subplot(2,3,1)
sns.boxplot(x='thal',data=dt)
plt.subplot(2,3,2)
sns.boxplot(x='oldpeak',data=dt)
plt.subplot(2,3,3)
sns.boxplot(x='thalach',data=dt)
plt.subplot(2,3,4)
sns.boxplot(x='chol',data=dt)
plt.subplot(2,3,5)
sns.boxplot(x='trestbps',data=dt)
plt.subplot(2,3,6)
sns.boxplot(x='age',data=dt)
plt.show()

In [ ]:
dt.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved',
       'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']

In [ ]:
dt.describe()

In [ ]:
ranges = [0, 120, 140, np.inf]
labels = [0, 1, 2]

dt['resting_blood_pressure'] = pd.cut(dt.resting_blood_pressure, bins=ranges, labels=labels)

In [ ]:
ranges = [0, 200, 270, np.inf]
labels = [0, 1, 2]

dt['cholesterol'] = pd.cut(dt.cholesterol, bins=ranges, labels=labels)

In [ ]:
ranges = [0, 130, 160, np.inf]
labels = [0, 1, 2]

dt['max_heart_rate_achieved'] = pd.cut(dt.max_heart_rate_achieved, bins=ranges, labels=labels)

In [ ]:
ranges = [0, 0.8, 1.6, np.inf]
labels = [0, 1, 2]

dt['st_depression'] = pd.cut(dt.st_depression, bins=ranges, labels=labels)

In [ ]:
dt['resting_blood_pressure'] = pd.to_numeric(dt.resting_blood_pressure)
dt['cholesterol'] = pd.to_numeric(dt.cholesterol)
dt['max_heart_rate_achieved'] = pd.to_numeric(dt.max_heart_rate_achieved)
dt['st_depression'] = pd.to_numeric(dt.st_depression)

In [ ]:
for i in dt.columns:
    dt[i].fillna(dt[i].mean(), inplace=True)

In [ ]:
dt.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dt.drop('target', 1), dt['target'], test_size = 0.3, random_state=10) #split the data

dtree = DecisionTreeClassifier()

para_dt = {'criterion':['gini','entropy'],'max_depth':np.arange(1, 50), 'min_samples_leaf':[1,2,4,5,10,20,30,40,80,100]}

grid = GridSearchCV(dtree, para_dt, cv=5)

grid.fit(X_train, y_train)

grid.best_params_

grid.best_score_

## Decision Tree

In [ ]:
dt_model = DecisionTreeClassifier(criterion='entropy', max_depth=6, min_samples_leaf=6)
dt_model.fit(X_train, y_train)
dt_model.score(X_test, y_test)

rf = RandomForestClassifier()

para_rf = {'criterion':['gini','entropy'],'max_depth':np.arange(1, 50), 'min_samples_leaf':[1,2,4,5,10,20,30,40,80,100]}

grid = GridSearchCV(dtree, para_dt, cv=5)

grid.fit(X_train, y_train)

grid.best_params_


## Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=300, criterion='entropy', max_depth= 3, min_samples_leaf= 5)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

## Boosting

In [ ]:
X, y = dt.iloc[:,:-1], dt.iloc[:,-1]

data_dmatrix = xgb.DMatrix(data=X,label=y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dmatrix = xgb.DMatrix(data=X,label=y)
gbm_param_grid = {'learning_rate': np.arange(0.05,1.05,.05),
                  'n_estimators': [100, 200, 500],
                  'subsample': np.arange(0.05,1.05,.05)}

gbm = xgb.XGBRFClassifier()

randomized_mse = RandomizedSearchCV(estimator=gbm, param_distributions=gbm_param_grid, 
                                    n_iter=25, cv=5, verbose=1)

randomized_mse.fit(X, y)

In [ ]:
clf = xgb.XGBClassifier(subsample=0.7, n_estimators= 500, learning_rate=1.0, num_boost_round=10)
clf.fit(X_train, y_train)
print('Score: ',clf.score(X_test, y_test))